In [91]:
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

import parse
import re


In [172]:
metric_name = "overall_acc"
df = pd.read_csv("raw_data.csv")
df = df.rename(columns={"Unnamed: 0": "index"})
# df = df.drop(columns=["Unnamed: 0"])
df = df.dropna(subset=[f"test_autoconstructive_{metric_name}"])
df_wlicoxon = pd.read_csv("analysis_wilcoxon.csv")


print(df.columns)

techniques = ["1-nn", "3-nn", "svm", "rf", "xgboost", "autoconstructive"]
num_techniques = len(techniques)
projects = df["project"].unique()
num_projects = len(projects)
# for technique in techniques:
# 	df[f"diff_test_val_{technique}"] = df[f"test_{technique}_{metric_name}"] - df[f"val_{technique}_{metric_name}"]
# 	df[f"diff_val_train_{technique}"] = df[f"val_{technique}_{metric_name}"] - df[f"train_{technique}_{metric_name}"]
# 	df[f"diff_test_train_{technique}"] = df[f"test_{technique}_{metric_name}"] - df[f"train_{technique}_{metric_name}"]
# 	df[f"diffs_{technique}"] = (df[f"diff_test_val_{technique}"].abs() + df[f"diff_val_train_{technique}"].abs() + df[f"diff_test_train_{technique}"].abs())/3


# df["project-dataset_name"] = df["project"] + "-" +df["dataset_name"]
df = df.sort_values(by=["dataset_name"])
df.head()

Index(['index', 'project', 'dataset_name',
       'train_autoconstructive_overall_acc', 'train_1-nn_overall_acc',
       'val_3-nn_overall_acc', 'test_1-nn_overall_acc',
       'val_xgboost_overall_acc', 'train_rf_overall_acc',
       'val_1-nn_overall_acc', 'train_3-nn_overall_acc',
       'train_xgboost_overall_acc', 'train_svm_overall_acc',
       'test_autoconstructive_overall_acc', 'val_svm_overall_acc',
       'test_rf_overall_acc', 'val_rf_overall_acc', 'test_3-nn_overall_acc',
       'test_svm_overall_acc', 'test_xgboost_overall_acc',
       'val_autoconstructive_overall_acc'],
      dtype='object')


,index,project,dataset_name,train_autoconstructive_overall_acc,train_1-nn_overall_acc,val_3-nn_overall_acc,test_1-nn_overall_acc,val_xgboost_overall_acc,train_rf_overall_acc,val_1-nn_overall_acc,...,train_xgboost_overall_acc,train_svm_overall_acc,test_autoconstructive_overall_acc,val_svm_overall_acc,test_rf_overall_acc,val_rf_overall_acc,test_3-nn_overall_acc,test_svm_overall_acc,test_xgboost_overall_acc,val_autoconstructive_overall_acc
3482,3482,exp0090_politica_5_oracle_1m5l_mon_metric_test...,Australian,0.854779,1.0,0.779412,0.852941,0.867647,1.0,0.720588,...,1.0,0.906250,0.911765,0.823529,0.867647,0.882353,0.852941,0.852941,0.926471,0.794118
3122,3122,exp0090_politica_4_diff_best_holdout_1m1l_nosbss,Australian,0.882246,1.0,0.898551,0.782609,0.898551,1.0,0.855072,...,1.0,0.876812,0.855072,0.869565,0.884058,0.927536,0.811594,0.840580,0.869565,0.898551
3121,3121,exp0090_politica_4_diff_best_holdout_1m1l_nosbss,Australian,0.880435,1.0,0.869565,0.811594,0.855072,1.0,0.826087,...,1.0,0.878623,0.855072,0.898551,0.855072,0.913043,0.855072,0.869565,0.840580,0.913043
1989,1989,exp0090_politica_2_holdout_1m5l_append_orginal...,Australian,0.990809,1.0,0.779412,0.852941,0.867647,1.0,0.720588,...,1.0,0.906250,0.838235,0.823529,0.867647,0.882353,0.852941,0.852941,0.926471,0.882353
177,177,exp0090_politica_1_oracle_1m1l,Australian,0.858456,1.0,0.867647,0.852941,0.867647,1.0,0.823529,...,1.0,0.911765,0.926471,0.882353,0.867647,0.882353,0.838235,0.823529,0.911765,0.867647


In [248]:
df_melted=pd.melt(df, id_vars=["project", "dataset_name", "index"])
df_melted["split"]=df_melted["variable"].apply(lambda x: re.search(r"(train|val|test).*", x).group(1))
df_melted["technique"]=df_melted["variable"].apply(lambda x: re.search(f"[train|val|test]_(.*)_{metric_name}.*", x).group(1))
df_melted = df_melted.drop(columns=["variable"])
df_melted = df_melted[(df_melted["technique"] == "autoconstructive") | (df_melted["project"] == "exp0090_politica_1_oracle_1m1l")]
df_melted["technique"] = df_melted.apply(lambda x: f'{x["technique"]}_{x["project"]}' if x["technique"] == "autoconstructive" else x["technique"], axis=1)
# df_melted = df_melted.dropna()
df_melted = df_melted.pivot(index=["project", "dataset_name", "technique", "index"], columns=["split"])
df_melted.columns = [col[0] if col[0] != "value" else col[1] for col in df_melted.columns.values]
df_melted = df_melted.reset_index()
df_melted = df_melted.set_index(["index"])
# print(df_melted.columns.values)
df_melted = df_melted
df_melted["mean_diff"] = ( 
	# (df_melted["train"]-df_melted["val"]).abs()# + 
	# (df_melted["train"]- df_melted["test"]).abs() + 
	(df_melted["val"]-df_melted["test"]).abs()
) / 1
df_melted.head()

# df_melted.shape
# df_melted.stack(0).head()

,project,dataset_name,technique,test,train,val,mean_diff
index,,,,,,,
128,exp0090_politica_1_oracle_1m1l,Australian,1-nn,0.852941,1.0,0.823529,0.029412
148,exp0090_politica_1_oracle_1m1l,Australian,1-nn,0.867647,1.0,0.838235,0.029412
152,exp0090_politica_1_oracle_1m1l,Australian,1-nn,0.867647,1.0,0.808824,0.058824
163,exp0090_politica_1_oracle_1m1l,Australian,1-nn,0.852941,1.0,0.852941,0.000000
177,exp0090_politica_1_oracle_1m1l,Australian,1-nn,0.852941,1.0,0.823529,0.029412


In [249]:
px.box(df_melted, x="technique", y="mean_diff", color="dataset_name", height=1000, width=2000)

In [251]:
df_melted.groupby("technique").mean()

,test,train,val,mean_diff
technique,,,,
1-nn,0.823936,0.995003,0.819189,0.049258
3-nn,0.818622,0.912681,0.826975,0.042804
autoconstructive_exp0090_politica_1_oracle_1m1l,0.901670,0.882259,0.871375,0.040872
autoconstructive_exp0090_politica_1_oracle_1m1l_nosbss,0.881627,0.829607,0.834134,0.054534
autoconstructive_exp0090_politica_1_oracle_1m5l_mon_metric_test,0.912059,0.869601,0.863801,0.052152
autoconstructive_exp0090_politica_1_oracle_1m5l_mon_metric_test_append_orginal_inputs,0.914186,0.875925,0.868429,0.049537
autoconstructive_exp0090_politica_2_holdout_1m1l,0.855644,0.920954,0.897406,0.050159
autoconstructive_exp0090_politica_2_holdout_1m1l_nosbss,0.828562,0.867223,0.860804,0.045416
autoconstructive_exp0090_politica_2_holdout_1m5l,0.854405,0.937936,0.892906,0.043011


In [78]:

fig = None
fig = make_subplots(rows=num_techniques+num_projects, cols=1)

i=1
for technique in techniques:
	if technique != "autoconstructive":
		df_tmp = df[df["project"] == projects[0]]
		x = "dataset_name"
		color = None

		for f in px.box(df_tmp, x=x,y=f"diffs_{technique}", color=color).data:
			fig.add_trace(f, row=i, col=1)
			fig.update_xaxes(title_text=technique, row=i, col=1)
			fig.update_yaxes(range=[0, 0.3])

		i += 1
	else:
		df_tmp = df
		x = "dataset_name"
		color = None

		for project in projects:
			df_tmp = df[df["project"] == project]

			for f in px.box(df_tmp, x=x,y=f"diffs_{technique}", color=color).data:
				fig.add_trace(f, row=i, col=1)
				fig.update_xaxes(title_text=project, row=i, col=1)
				fig.update_yaxes(range=[0, 0.3])
				i += 1


fig.update_layout(margin=dict(l=20, r=20, t=200, b=200) , width=1400, height=3000)
fig.show()